# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_modesB.ipynb

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from earthfun import earthfun
from scipy.optimize import brentq
from surf_stress import surf_stress

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 12, 10
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# CONSTANTS

earthr   = 6371000.0
RSPAN    = np.array([3480000.0, earthr])
earthrkm = earthr/1e3

# Earth model (uniform mantle shell)
imod     = 0
RHO      = 4380                                                # density
MU       = 5930*5930*RHO                                       # rigidity (MU = 1.54e11)
beta     = np.sqrt(MU/RHO)

# frequencies
fmin     = 1/3600                                              # the initial frequency to start (period = one hour)
df       = 0.0002                                              # the frequency step size (chosen by trial and error)
fmax     = 0.08                                                # stopping frequency (arbitrary)
fvec     = np.arange(fmin,fmax,df)
numf     = len(fvec)

# max mode branch (nmax = 3 means 4 branches: n=0,1,2,3)
nmax     = 3

# degrees (l)
lvec     = np.arange(1,11)
#lvec     = np.array([40,0]); nmax = 0                          # for checking the system of equations (at the end)
nl       = len(lvec)

MAX_STEP = 5e4                                                 # Step size for ODE solver in surf_stress()

In [ ]:
# initialize roots (one set per earth model)
# this will collect each froots vector
fgrid = np.full((nmax+1,nl),np.nan)

for hh in range(nl):                                           # loop over the angular order
    l = lvec[hh]
    print('Degree (l) = %i'% (l))

    if l < 1:
        print('Error: l = 1 is the minimum allowable value. Skipping')
        continue 

    ii = 0
    f = fvec[ii]
    Tsurf = surf_stress(f, l, imod, RHO, MU, RSPAN, MAX_STEP)
    n = 0                                                      # counter for n (n=0 is first root)

    print('freq %3i/%3i %10.3e %10.3e %.2f mHz %.1f s %.2f min'% (ii+1, numf, np.nan, Tsurf, f*1e3, 1/f, 1/f/60))

    # leave gap for T(n=0,l=1), which do not exist
    # note: these are useful when looping over degree l
    if l == 1 and nmax > 0:                                    # fill the n >= 1 entries
        n = 1

    if l == 1 and nmax == 0:                                   # exit loop early (mode 0T1 does not exist)
        continue

    for ii in range(1,len(fvec)):                              # loop over the frequencies
        oldf = f
        f = fvec[ii]

        # compute the surface stress for this frequency
        Tsurfold = Tsurf
        Tsurf = surf_stress(f, l, imod, RHO, MU, RSPAN, MAX_STEP)

        print('freq %3i/%3i %10.3e %10.3e %.2f mHz %.1f s %.2f min'% (ii+1, numf, Tsurfold, Tsurf, f*1e3, 1/f, 1/f/60))

        if Tsurfold * Tsurf < 0:
            f0 = brentq(surf_stress,oldf,f, args=(l, imod, RHO, MU, RSPAN, MAX_STEP))
            #froots[n,0] = f0
            fgrid[n,hh] = f0
            
            print('T(earthr)=0 --> n=%i %.3f mHz l=%i'% (n,f0*1e3,l))

            # update eigenfunctions (WT, rvec) for the exact frequency
            _, WT, rvec = surf_stress(f0, l, imod, RHO, MU, RSPAN, MAX_STEP, return_wt_rvec=True)

            if n == nmax:                                      # this halts the search after nmax roots
                break

            # advance to the next root (i.e., overtone)
            n = n+1

In [ ]:
# mesh for plotting
L, N = np.meshgrid(lvec,np.arange(0,nmax+1))

# dispersion quantities
Tgrid = 1/fgrid
kgrid = np.sqrt(L*(L+1))
cgrid = 2*np.pi/np.sqrt(L*(L+1))*fgrid
wgrid = 2*np.pi*fgrid
lamgrid = 2*np.pi/kgrid

title = 'Toroidal modes for spherical shell: \n homo, rho = %.0f kg/m3, mu = %.3e Pa (white)'% (RHO,MU)

In [ ]:
# dispersion plot A

plt.figure()
plt.plot(L.T, fgrid.T*1E3,'k-',lw=2)
plt.plot(L, fgrid*1E3,'ko',ms=10,mfc='w')
plt.axis([0, 11, 0, 4])
plt.gca().set(xlabel='Degree (l)', ylabel='Frequency (f), mHz', title=title)
plt.show()

In [ ]:
# dispersion plot B

plt.figure()
plt.plot(Tgrid.T, cgrid.T*earthrkm,'k-',lw=2)
plt.plot(Tgrid, cgrid*earthrkm,'ko',ms=10,mfc='w')
plt.grid(visible=True)
#plt.axis([0., 300., 3.5, 7.])
plt.gca().set(xlabel='Period (T), s', ylabel='Phase speed (c), km/s', title=title)
plt.show()

In [ ]:
# dispersion plot C

plt.figure()
plt.plot(kgrid.T/earthrkm, wgrid.T,'k-',lw=2)
plt.plot(kgrid/earthrkm, wgrid,'ko',ms=10,mfc='w')
plt.gca().set(xlabel='Wavenumber (k), 1/km', ylabel='Angular frequency (ω), rad/s', title=title)
plt.show()

In [ ]:
# check the system of equations
# to check, change the False flag to True, then uncomment the line at the top: "lvec     = np.array([40,0]); nmax = 0"
# this will check for 0T40 (n=0, l=40)

if False:
    # eigenfunction
    Wr = WT[0,:]
    Tr = WT[1,:]
    drvec = np.diff(rvec)
    drvec = np.append(drvec[0],drvec)

    l = lvec[0]
    rhovec = 0*Wr + RHO
    muvec  = 0*Wr + MU
    
    print('CHECKING THE SYSTEM OF EQUATIONS')
    omega0 = 2*np.pi*f0
    print('imod =\n', imod)
    print(np.sum( rhovec * Wr**2 * rvec**2 * drvec ))                          # check
    print('l = %i, n = 0, T = %.1f s, f = %.2f mHz, ω = %.2e 1/s, ω^2 = %.2e 1/s'% (l,1/f0,f0*1E3,omega0,omega0**2))
    
    L1 = np.gradient(Wr,rvec)                                                  # left - top
    R1 = np.add(Wr/rvec, Tr/muvec)                                             # right - top
    L2 = np.gradient(Tr,rvec)                                                  # left - bottom
    R2 = (-(2*np.pi*f0)**2*rhovec + (l+2)*(l-1)/rvec**2*muvec)*Wr - 3/rvec*Tr  # right - bottom
    
    print('checking the 1st equation:')
    print(np.linalg.norm(R1-L1) / np.linalg.norm(R1))
    print('checking the 2st equation:')
    print(np.linalg.norm(R2-L2) / np.linalg.norm(R2))
    print('checking the boundary condition:')
    print(Tr[0]/max(abs(Tr)))
    print(Tr[-1]/max(abs(Tr)))

    fig, ax = plt.subplots(nrows=3,ncols=2)
    ax[0,0].plot(muvec,rvec,'k.')
    ax[0,0].set(xlabel='mu(r), Pa',ylabel='radius, km')
    ax[0,1].plot(rhovec,rvec,'k.')
    ax[0,1].set(xlabel='rho(r), kg/m^3',ylabel='radius, km')
    ax[1,0].plot(Wr,rvec,'b.')
    ax[1,0].set(xlabel='W(r)')
    ax[1,1].plot(Tr,rvec,'r.')
    ax[1,1].set(xlabel='T(r)')
    ax[2,0].plot(np.gradient(Wr,rvec),rvec,'b.')
    ax[2,0].set(xlabel='dW/dr')
    ax[2,1].plot(np.gradient(Tr,rvec),rvec,'r.')
    ax[2,1].set(xlabel='dT/dr')

    fig, ax = plt.subplots(nrows=2,ncols=1)
    axmn = min(np.minimum(R1,L1))
    axmx = max(np.maximum(R1,L1))
    ax[0].plot([axmn,axmx],[axmn,axmx],'k--')
    ax[0].plot(R1,L1,'b.')
    ax[0].set(title='check for dW/dr')
    ax[0].grid(visible=True)
    ax[0].axis('equal')
    axmn = min(np.minimum(R2,L2))
    axmx=max(np.maximum(R2,L2))
    ax[1].plot([axmn,axmx],[axmn,axmx],'k--')
    ax[1].plot(R2,L2,'r.')
    ax[1].set(title='check for dT/dr')
    ax[1].grid(visible=True)
    ax[1].axis('equal')
    plt.show()